# Model Notebook

The notebook for creating age gender detection model. Here, we will extract the data, train it and fit it into our model and will evaluate our results and optimize the model for better results.

## Importing Necessary Libraries

In [1]:
import tensorflow as tf
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, Adagrad, Adadelta, Adamax, RMSprop

## Extract Data From Dataset

In [2]:
import os
fldr = "C:/Users/ravi ranjan verma/OneDrive/Desktop/AgeGenderDetection/UTKFace"

In [3]:
files = os.listdir(fldr)

## Create List of Images From The Dataset

In [ ]:
ages = []
genders = []
images = []

for fle in files:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    total=fldr+'/'+fle
    print(total)
    image=cv2.imread(total)
    image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(48,48))
    images.append(image)

## Create List Of Age and Gender From The Datset

In [5]:
for fle in files:
    age=int(fle.split('_')[0])
    gender=int(fle.split('_')[1])
    ages.append(age)
    genders.append(gender)

In [ ]:
plt.imshow(images[96])

In [ ]:
print(ages[96])
print(genders[96])

## Saving Images, Ages, Genders as Array in the sys.

In [8]:
images_f = np.array(images)
ages_f = np.array(ages)
genders_f = np.array(genders)

In [ ]:
np.save(fldr+'image.npy',images_f)
np.save(fldr+'ages.npy',ages_f)
np.save(fldr+'genders.npy',genders_f)

## Finding the No. of Male & Female Samples Respectivily

In [ ]:
values, counts = np.unique(genders_f, return_counts=True)
print(counts)

## Plotting the No. of Male & Female Samples

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
gender = ['Male', 'Female']
values = [12391, 11317]
ax.bar(gender, values)
plt.show()

## Finding the No. of samples in each age

In [ ]:
values, counts = np.unique(ages_f, return_counts=True)
print(counts)

In [13]:
val = values.tolist()
cnt = counts.tolist()

## Plotting the sample agewise

In [ ]:
plt.plot(counts)
plt.xlabel('Ages')
plt.ylabel('Distribution')
plt.show()

## Define LAbel For Output

In [15]:
labels = []
i = 0
while i<len(ages):
    label=[]
    label.append(ages[i])
    label.append(genders[i])
    labels.append(label)
    i=i+1

## Creating Another List of Img From existing-0ne
For Better Learning

In [16]:
images_f_2 = images_f/255

In [ ]:
images_f_2.shape

In [18]:
labels_f = np.array(labels)

## Splitting The Dataset Into TEst-Train

In [ ]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(images_f_2,labels_f,test_size=0.3)

In [ ]:
Y_train[0:5]

## Splitting Y_test & Y_train Further
### For better understanding

In [22]:
Y_train_2 = [Y_train[:,1],Y_train[:,0]]
Y_test_2 = [Y_test[:,1],Y_test[:,0]]

In [ ]:
Y_train_2[0][0:5]

In [ ]:
Y_test_2[1][0:5]

## Define The Model

In [25]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

def Convolution(input_tensor, filters):
    x = Conv2D(filters=filters, kernel_size=(3,3), padding="same", strides=(1,1), kernel_regularizer=l2(0.001))(input_tensor)
    x = Dropout(0.1)(x)
    x = Activation('relu')(x)
    return x

def model(input_shape):
    inputs = Input(shape=(input_shape))
    
    conv_1 = Convolution(inputs, 32)
    maxp_1 = MaxPooling2D(pool_size=(2, 2))(conv_1)
    
    conv_2 = Convolution(maxp_1, 64)
    maxp_2 = MaxPooling2D(pool_size=(2, 2))(conv_2)
    
    conv_3 = Convolution(maxp_2, 128)
    maxp_3 = MaxPooling2D(pool_size=(2, 2))(conv_3)
    
    conv_4 = Convolution(maxp_3, 256)
    maxp_4 = MaxPooling2D(pool_size=(2, 2))(conv_4)
    
    flatten = Flatten()(maxp_4)
    
    dense_1 = Dense(64, activation='relu')(flatten)
    drop_1 = Dropout(0.2)(dense_1)
    output_1 = Dense(1, activation='sigmoid', name='sex_out')(drop_1)
    
    dense_2 = Dense(64, activation='relu')(flatten)
    drop_2 = Dropout(0.2)(dense_2)
    output_2 = Dense(1, activation='relu', name='age_out')(drop_2)
    
    model = Model(inputs=[inputs], outputs=[output_1, output_2])
    model.compile(loss=["binary_crossentropy", "mae"], optimizer="Adam", metrics=[["accuracy"], ["accuracy"]])
    
    return model


In [26]:
Model=model((48,48,3))

In [ ]:
Model.summary()

In [28]:
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

## Initializing The Model

In [29]:
fle_s='Age_Sex_Detection.keras'
checkpoint=ModelCheckpoint(fle_s, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75, monitor='val_loss',restore_best_weights=True)
callback_list=[checkpoint,Early_stop]

## Running The Model
### This will take some time

In [ ]:
History=Model.fit(
    X_train,Y_train_2,
    batch_size=64,
    validation_data=(X_test,Y_test_2),
    epochs=25,
    callbacks=callback_list,
    )

## Model Evaluation

In [ ]:
Model.evaluate(X_test,Y_test_2)

In [ ]:
pred=Model.predict(X_test)

In [ ]:
pred[1]

## Plotting Loss

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.xlabel=('Epoch')
plt.ylabel=('Loss')
plt.legend(['Train','Validation'],loc='upper right')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

## Plotting Sex Accuracy

In [ ]:
plt.plot(History.history['sex_out_accuracy'])
plt.plot(History.history['val_sex_out_accuracy'])
plt.title('Model Accuracy')
plt.xlabel=('Epoch')
plt.ylabel=('Accuracy')
plt.legend(['Train','Validation'],loc='upper right')
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

## Plotting Age Prediction

In [ ]:
fig,ax=plt.subplots()
ax.scatter(Y_test_2[1],pred[1])
ax.plot([Y_test_2[1].min(),Y_test_2[1].max()],[Y_test_2[1].min(),Y_test_2[1].max()],'k--',lw=4)
ax.set_xlabel('Actual Age')
ax.set_ylabel('Predicted Age')
plt.show()

### Making Report of Model

In [ ]:
i=0
Pred_l=[]
while(i<len(pred[0])):
    Pred_l.append(int(np.round(pred[0][i])))
    i=i+1

In [45]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
report=classification_report(Y_test_2[0],Pred_l)
print(report)

### Making Heatmap of the Model(Gender only)

In [ ]:
results=confusion_matrix(Y_test_2[0],Pred_l)

In [ ]:
import seaborn as sns
sns.heatmap(results,annot=True)

### Testing the images ourself

In [50]:
def test_image(ind,images_f,images_f_2,Model):
    plt.imshow(images_f[ind])
    image_test=images_f_2[ind]
    Pred_l=Model.predict(np.array([image_test]))
    sex_f=['Male','Female']
    age=int(np.round(Pred_l[1][0]))
    sex=int(np.round(Pred_l[0][0]))
    print("Predicted Age is "+ str(age))
    print("Predicted Gender is "+ sex_f[sex])

### Test-1

In [ ]:
test_image(4,images_f,images_f_2,Model)

### Test-2

In [ ]:
test_image(94,images_f,images_f_2,Model)

### Test-3
Error in the Age

In [ ]:
test_image(954,images_f,images_f_2,Model)

### Test-4

In [ ]:
test_image(1424,images_f,images_f_2,Model)